In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium. webdriver.common.by import By


def get_links(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get("https://www.immoweb.be/en")
    
    # Handle cookie consent
    shadow_host = driver.find_element(By.ID, 'usercentrics-root')
    root = driver.execute_script('return arguments[0].shadowRoot', shadow_host)
    cookie_button = root.find_element(By.CSS_SELECTOR, '[data-testid=uc-accept-all-button]')
    cookie_button.click()

    # Search for properties
    driver.get(url)
    
    # Request page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract links to property listings
    links = [elem.get("href") for elem in soup.find_all("a", attrs={"class": "card__title-link"})]
    if not links:
        print("No properties found.")
        return None
    
    df = pd.DataFrame(links,columns=["Property Links"])
    df.to_csv('Links_immo.csv', index=False)

    print("Links saved to Links.csv")
search_url = "https://www.immoweb.be/en/search/house/for-sale?countries=BE&page=1&orderBy=relevance"
property_data = get_links(search_url)


Links saved to Links.csv


In [ ]:
import asyncio
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv

# Asyncio function for HTML
async def fetch_url(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))  
    driver.get(url)
    await asyncio.sleep(3)  
    html = driver.page_source
    driver.quit()
    return html

# Asynco function 
async def process_property_info(url):
    html = await fetch_url(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    
    property_id = soup.find('div', class_="classified__header--immoweb-code").text.strip() if soup.find('div', class_="classified__header--immoweb-code") else None
    price = soup.find('p', class_="classified__price").find('span', {'aria-hidden': 'true'}).text.strip() if soup.find('p', class_="classified__price") else None
    address_full = soup.find_all('span', class_="classified__information--address-row")
    postal_code, locality = address_full[1].text.split('—') if len(address_full) > 1 else (None, None)
    number_of_rooms = soup.find('span', class_='overview__text').text.strip() if soup.find('span', 'overview__text') else None
    
   
    rows = soup.find_all('tr', class_='classified-table__row')
    living_area = None
    for row in rows:
        header = row.find('th')
        if header and "Living area" in header.text:
            living_area = row.find('td').text.strip() if row.find('td') else None
            living_area = living_area.replace('m²', '').strip() if living_area else None
            break
    
    
    return {
        'Property ID': property_id,
        'Locality name': locality.strip() if locality else None,
        'Postal code': postal_code.strip() if postal_code else None,
        'Price': price,
        'Number of rooms': number_of_rooms,
        'Living area (m²)': living_area
    }

# Main function
async def process_all_properties(urls):
    tasks = [process_property_info(url) for url in urls]
    properties = await asyncio.gather(*tasks)  # All processes in one

    # Saved results CSV
    with open('properties_async.csv', 'w', newline='',encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['Property ID', 'Locality name', 'Postal code', 'Price', 'Number of rooms', 'Living area (m²)'])
        writer.writeheader()
        writer.writerows(properties)
    
    print(f"Saved info about {len(properties)} properties у properties_async.csv")


urls = ["https://www.immoweb.be/en/classified/house/for-sale/kortrijk/8500/20229361", "https://www.immoweb.be/en/classified/house/for-sale/antwerp/2000/12345678"]  

#
await process_all_properties(urls)

In [ ]:
import asyncio
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Read URL from CSV
urls = []
with open('property_links_filtered.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # Pass header
    for row in reader:
        urls.append(row[0])  # Add first row to list

# Create a semaphore with a limited requests
semaphore = asyncio.Semaphore(10)

# Asynchronous function to get HTML with WebDriverWait
async def fetch_url(url):
    async with semaphore:  # Acquire the semaphore before running the code block
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))  
        driver.get(url)
        
        try:
            # Очікуємо появи потрібного елемента на сторінці (наприклад, елемента з ID або класом)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "classified__header--immoweb-code"))
            )
        except Exception as e:
            print(f"Error loading page {url}: {e}")
            driver.quit()
            return None  # Якщо сторінка не завантажилась, повертаємо None
        
        # Отримуємо HTML після того, як сторінка повністю завантажена
        html = driver.page_source
        driver.quit()
        return html

# An asynchronous function for processing pages
async def process_property_info(url):
    html = await fetch_url(url)
    if html is None:
        return None  # Якщо fetch_url повернув None, пропускаємо цей запис
    
    soup = BeautifulSoup(html, 'html.parser')
    
    # Get data
    property_id = soup.find('div', class_="classified__header--immoweb-code").text.strip() if soup.find('div', class_="classified__header--immoweb-code") else None
    price = soup.find('p', class_="classified__price").find('span', {'aria-hidden': 'true'}).text.strip() if soup.find('p', class_="classified__price") else None
    address_full = soup.find_all('span', class_="classified__information--address-row")
    postal_code, locality = address_full[1].text.split('—') if len(address_full) > 1 else (None, None)
    number_of_rooms = soup.find('span', class_='overview__text').text.strip() if soup.find('span', 'overview__text') else None
    
    # Find living area
    rows = soup.find_all('tr', class_='classified-table__row')
    living_area = None
    for row in rows:
        header = row.find('th')
        if header and "Living area" in header.text:
            living_area = row.find('td').text.strip() if row.find('td') else None
            living_area = living_area.replace('m²', '').strip() if living_area else None
            break
    
    # Return collected info
    return {
        'Property ID': property_id,
        'Locality name': locality.strip() if locality else None,
        'Postal code': postal_code.strip() if postal_code else None,
        'Price': price,
        'Number of rooms': number_of_rooms,
        'Living area (m²)': living_area
    }

# Main function for processing all URLs
async def process_all_properties(urls):
    tasks = [process_property_info(url) for url in urls]
    properties = await asyncio.gather(*tasks)  # run all tasks at the same time

    # Filter out None results (in case some properties were skipped)
    properties = [prop for prop in properties if prop is not None]
    
    # We save the results in CSV with the correct encoding
    with open('properties_async.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['Property ID', 'Locality name', 'Postal code', 'Price', 'Number of rooms', 'Living area (m²)'])
        writer.writeheader()
        writer.writerows(properties)
    
    print(f"Saved data of {len(properties)} properties in properties_async.csv")

# Jupyter Notebook doesn't use asyncio.run(), just await
await process_all_properties(urls)
